https://oauth.vk.com/authorize?client_id=6121396&scope=1&redirect_uri=https://oauth.vk.com/blank.html&display=page&response_type=token&revoke=1

In [1]:
pip install cython pillow>=7.0.0 numpy>=1.18.1 opencv-python>=4.1.2 torch>=1.9.0 --extra-index-url https://download.pytorch.org/whl/cpu torchvision>=0.10.0 --extra-index-url https://download.pytorch.org/whl/cpu pytest==7.1.3 tqdm==4.64.1 scipy>=1.7.3 matplotlib>=3.4.3 mock==4.0.3

zsh:1: 7.0.0 not found
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imageai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
group_key = 'vk1.a.TeWi79plYP3gMA3B-1YZYQ2UAPXpM_9yz_tkhtYF-aylOGlu5xwqr2TIABetNarvAQNp06nFIZ3zR_WHcvjVQasQqb_77AzttZzynWdfCK5VqG3gLk8qz4wDbsej07I70-WHTnrxHeOm30op9d2e90GA1PovKXgTKMGIP2CXObX5RueDkXmjycupY6K5ZgeUcOYUhleCWIdV0RoPhcy5CA'

In [23]:
import vk_api 
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id

import requests, io, cv2
import numpy as np
from PIL import Image

In [25]:
from typing import List
from PIL import Image


def get_url(person_info: List) -> str:
    return person_info[0]["crop_photo"]['photo']['sizes'][-1]["url"]


def get_np_image_from_url(url: str) -> np.ndarray:
    response = requests.get(url)
    bytes_im = io.BytesIO(response.content)
    cv_im = np.array(Image.open(bytes_im))

    return cv_im


def detect(np_image: np.ndarray) -> np.ndarray:
    returned_image, detections, extracted_objects = detector.detectObjectsFromImage(
        input_image=np_image,
        output_type="array",
        extract_detected_objects=True, 
        minimum_percentage_probability=30)

    return returned_image


def save_np_image(np_image: np.ndarray, save_path: str = "image.jpg") -> None:
    im = Image.fromarray(np_image)
    im.save(save_path)    


def send_image(path_to_image: str = "image.jpg"):
    photo = upload.photo_messages('image.jpg')
    owner_id = photo[0]['owner_id']
    photo_id = photo[0]['id']
    access_key = photo[0]['access_key']
    attachment = f'photo{owner_id}_{photo_id}_{access_key}'
    vk.messages.send(                
        user_id=event.user_id,
        random_id=get_random_id(),
        attachment=attachment
    )

In [26]:
from imageai.Detection import ObjectDetection

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("yolov3.pt")
detector.loadModel()

In [30]:
vk_session = vk_api.VkApi(token = group_key)
upload = vk_api.VkUpload(vk_session)

longpoll = VkLongPoll(vk_session)
vk = vk_session.get_api()

for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW and event.to_me:
        print(event.text)
        if (event.text.startswith('wall_id')):
            id = int(event.text.split()[1])
            person_info = vk.users.get(user_id=id, fields='crop_photo')
            url = get_url(person_info)
            np_image = get_np_image_from_url(url)
            processed_image = detect(np_image)
            save_np_image(processed_image)
            send_image()

        elif (event.text.startswith('post_id')):
            id_type, post_id, command = event.text.split()
            if event.text.endswith('image'):
                wall_info = vk.wall.get(user_id=id, fields='crop_photo')
            elif event.text.endswith('text'):
                pass
            else: 
                vk.messages.send(                
                    user_id=event.user_id,
                    random_id=get_random_id(),
                    message=f"unknown command {command}"
                )


post_id 1 hui


ReadTimeout: HTTPSConnectionPool(host='im.vk.com', port=443): Read timed out. (read timeout=35)